In [ ]:
'''Objective: '''
import pandas as pd
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
"""Get boot reports"""
import requests
from urlparse import urljoin
import kernelci_api_key

BACKEND_URL = "http://api.kernelci.org"
JOB = 'mainline'
BOARD = 'am335x-boneblack'
DEFCONFIG = 'multi_v7_defconfig'
DATE_RANGE = 275
STORAGE_SERVER = 'http://storage.kernelci.org'

def invoke():
    headers = { "Authorization": kernelci_api_key.getkernelcikey()}
    params = {
        "job": JOB,
        "board": BOARD,
        "defconfig":DEFCONFIG,
        "date_range": DATE_RANGE
    }
    url = urljoin(BACKEND_URL, "/boot")
    response = requests.get(url, params=params, headers=headers)
    return response.content

In [ ]:
content = invoke()
content

In [ ]:
contentjs = json.loads(content)
df = json_normalize(contentjs['result'])
df.head()

In [ ]:
df.columns

In [ ]:
df2 = df[[u'_id.$oid',u'boot_result_description',u'created_on.$date',u'defconfig',u'git_describe',u'lab_name',u'status',u'time.$date',u'warnings']]
df2.head()

In [ ]:
df2.columns

In [ ]:
df2.columns = [u'boot_id',u'boot_result_description', u'created_on', u'defconfig', u'git_describe', u'lab_name', u'status',u'boot_time',u'warnings']
df2.head()

In [ ]:
df2['created_on'] = pd.to_datetime(df2['created_on'],unit='ms')
df2.head()

In [ ]:
df2.describe(include='all')

In [ ]:
df2.index = df2.created_on
df2.head()

In [ ]:
df2 = df2.sort_index()

In [ ]:
df2.status.value_counts()

In [ ]:
df2 = df2[(df2.status == 'PASS') | (df2.status == 'FAIL')]
df2.status.value_counts()

In [ ]:
df2.head()

In [ ]:
df2['status_fl'] = df2.status.apply(lambda x: 1 if x=='PASS' else 0)
df2.head()

In [ ]:
df3 = df2.resample('W', how='mean')
df3.head()

In [ ]:
df3.status_fl.plot()
plt.ylim(0.4,1)

In [ ]:
df3.boot_time.plot()

In [ ]:
#Select only boot_time (in secs) and git_describe

#Ignore cases where boot time is 0
df4 = df2[df2.boot_time != 0]
df4 = df4[['boot_time', 'git_describe']]
df4['boot_time'] = (df4['boot_time']/1000).astype(int)

In [ ]:
#Plot above with interaction

from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.models import ColumnDataSource, Circle, HoverTool, CustomJS

(x,y,k) = (df4.index.to_series(), df4['boot_time'], df4['git_describe'])

#Have to do this as Bokeh can print Dates properly on Hoever
ts = pd.to_datetime(x.values) 
z = ts.strftime('%Y.%m.%d')


output_notebook()
p = figure(width=600, height=300, x_axis_type='datetime',title='boot time')
p.line(x, y, line_dash="4 4", line_width=2, color='gray')

source = ColumnDataSource({'x':x, 'y':y, 'z':z, 'k':k})

invisible_circle = Circle(x='x',y='y', fill_color='gray', fill_alpha=0.1, line_color=None, size=10)
visible_circle = Circle(x='x',y='y', fill_color='firebrick', fill_alpha=0.5, line_color=None, size=10)

cr = p.add_glyph(source, invisible_circle, selection_glyph=visible_circle, nonselection_glyph=invisible_circle)

#Add hover tool, that selects the circle
# Add a hover tool, that selects the circle
code = "source.set('selected', cb_data['index']);"

callback = CustomJS(args={'source': source}, code=code)

p.add_tools(HoverTool(
                tooltips=[
                            ("d", "@z"),
                            ("bt", "@y"),
                            ('gd', "@k")
                        ], 
                callback=callback, 
                renderers=[cr], 
                mode='vline'))


show(p)